#  Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import yahoo_fin
import datetime
from datetime import date
import yahoo_fin.stock_info as yfsi
import re

In [1]:
import lib.contentBasedRecom
from lib.contentBasedRecom import createNames, colsForMatrix, returnColumn, createDf, getRecommendations

In [9]:
pd.DataFrame.from_dict(pref_stock_dict, orient='index').transpose()

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
0            0.0            0.0           0.0          5.063   

   operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
0               0.0             NaN             NaN         1.853   

   returnOnEquity  enterpriseToEbitda  priceToBook  \
0             NaN                 NaN     5.373768   

   priceToSalesTrailing12Months  forwardPE  trailingPE  
0                           NaN        NaN         NaN

In [10]:
pref_stock_dict

{'ebitdaMargins': 0,
 'profitMargins': 0,
 'grossMargins': 0,
 'revenueGrowth': 5.063,
 'operatingMargins': 0,
 'earningsGrowth': None,
 'returnOnAssets': None,
 'debtToEquity': 1.853,
 'returnOnEquity': None,
 'enterpriseToEbitda': None,
 'priceToBook': 5.3737683,
 'priceToSalesTrailing12Months': None,
 'forwardPE': None,
 'trailingPE': nan}

In [18]:
my_df = pd.read_csv('my_df', sep='\t')
my_df = my_df.drop(['Unnamed: 0'], axis=1)

In [20]:
my_df.head()

Symbol  ebitdaMargins|0.0  ebitdaMargins|0.1  ebitdaMargins|0.2  \
0    MMM                0.0                0.0                1.0   
1    ABT                0.0                0.0                1.0   
2   ABBV                0.0                0.0                0.0   
3   ABMD                0.0                0.0                1.0   
4    ACN                0.0                1.0                0.0   

   ebitdaMargins|0.3  ebitdaMargins|0.4  ebitdaMargins|0.5  ebitdaMargins|0.6  \
0                0.0                0.0                0.0                0.0   
1                0.0                0.0                0.0                0.0   
2                0.0                0.0                1.0                0.0   
3                0.0                0.0                0.0                0.0   
4                0.0                0.0                0.0                0.0   

   ebitdaMargins|0.7  ebitdaMargins|0.8  ebitdaMargins|0.9  ebitdaMargins|1.0  \
0                0.0                0.0                0.0                0.0   
1                0.0                0.0                0.0                0.0   
2                0.0                0.0                0.0                0.0   
3                0.0                0.0                0.0                0.0   
4                0.0                0.0                0.0                0.0   

   profitMargins|0.0  profitMargins|0.1  profitMargins|0.2  profitMargins|0.3  \
0                0.0                1.0                0.0                0.0   
1                0.0                1.0                0.0                0.0   
2                0.0                1.0                0.0                0.0   
3                0.0                1.0                0.0                0.0   
4                0.0                1.0                0.0                0.0   

   profitMargins|0.4  profitMargins|0.5  profitMargins|0.6  profitMargins|0.7  \
0                0.0                0.0                0.0                0.0   
1                0.0                0.0                0.0                0.0   
2                0.0                0.0                0.0                0.0   
3                0.0                0.0                0.0                0.0   
4                0.0                0.0                0.0                0.0   

   profitMargins|0.8  profitMargins|0.9  profitMargins|1.0  grossMargins|0.0  \
0                0.0                0.0                0.0               0.0   
1                0.0                0.0                0.0               0.0   
2                0.0                0.0                0.0               0.0   
3                0.0                0.0                0.0               0.0   
4                0.0                0.0                0.0               0.0   

   grossMargins|0.1  grossMargins|0.2  grossMargins|0.3  grossMargins|0.4  \
0               0.0               0.0               0.0               1.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               1.0               0.0   

   grossMargins|0.5  grossMargins|0.6  grossMargins|0.7  grossMargins|0.8  \
0               0.0               0.0               0.0               0.0   
1               1.0               0.0               0.0               0.0   
2               0.0               0.0               1.0               0.0   
3               0.0               0.0               0.0               1.0   
4               0.0               0.0               0.0               0.0   

   grossMargins|0.9  grossMargins|1.0  revenueGrowth|0.0  revenueGrowth|5.0  \
0               0.0               0.0                1.0                0.0   
1               0.0               0.0                1.0                0.

In [73]:
my_df_extra = pd.read_csv('my_df_extra', sep='\t')
my_df_extra = my_df_extra.drop(['Unnamed: 0'], axis=1)
my_df_extra.drop(['Security'], axis=1, inplace=True)
my_df_extra.set_index('Symbol', inplace=True)
my_df_extra.fillna(0, inplace=True)

In [21]:
my_df_extra.head()

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
Symbol                                                              
MMM           0.28613        0.16764       0.48024          0.071   
ABT           0.29229        0.17122       0.56850          0.234   
ABBV          0.52303        0.13656       0.70193          0.112   
ABMD          0.29598        0.15319       0.81897          0.183   
ACN           0.16804        0.11533       0.32348          0.272   

        operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
Symbol                                                                   
MMM              0.23218           0.000         0.10945       130.812   
ABT              0.20906           0.696         0.07740        53.112   
ABBV             0.36702           0.380         0.08480       594.660   
ABMD             0.26919          -0.088         0.11463         0.567   
ACN              0.15195           0.197         0.12456        16.607   

        returnOnEquity  enterpriseToEbitda  priceToBook  \
Symbol                                                    
MMM            0.44800              11.447     7.082453   
ABT            0.21862              20.950     7.199733   
ABBV           0.52255              10.608    17.558983   
ABMD           0.11621              54.203    11.782005   
ACN            0.31948              28.889    12.924301   

        priceToSalesTrailing12Months  forwardPE  trailingPE  
Symbol                                                       
MMM                         2.899046  17.104908   17.576897  
ABT                         5.857659  29.443275   34.776672  
ABBV                        4.312898   9.634188   32.075690  
ABMD                       16.754870  71.566864  110.085970  
ACN                         4.898481  39.977190   43.291637

In [22]:
# import index
data_sp500 = yfsi.tickers_sp500(True)
data_sp500 = data_sp500.rename(columns={'GICS Sector': 'Sector'})

## I normalized data first as fundamentals can be very different in magnitude. I will then first learn funk SVD on normalized data and re-scale it in the end

In [78]:
my_df_extra_normalized = (my_df_extra - np.mean(my_df_extra, axis=0)) / np.std(my_df_extra, axis=0)

In [80]:
my_df_extra_normalized

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
Symbol                                                              
MMM          0.207796       0.122045      0.185216      -0.147822   
ABT          0.242633       0.141022      0.567126      -0.096811   
ABBV         1.547550      -0.042711      1.144491      -0.134991   
ABMD         0.263501       0.045445      1.650936      -0.112772   
ACN         -0.460045      -0.155253     -0.493101      -0.084919   
ATVI         0.718192       0.781759      1.250679      -0.151578   
ADBE         0.946838       0.852739      1.923024      -0.107451   
AMD         -0.106981       0.650028      0.133204      -0.001047   
AAP         -0.791958      -0.454125      0.058215      -0.160340   
AES          0.533884      -0.504273     -0.754372      -0.109642   
AFL         -0.010501       0.228171     -0.135985      -0.193826   
A            0.137048       0.248421      0.440212      -0.132801   
APD          0.603332       0.311292     -0.577956      -0.099628   
AKAM         0.474559       0.175214      0.847739      -0.143441   
ALK         -1.827454      -0.753050     -1.624769       0.388892   
ALB         -0.028541      -0.427196     -0.550002      -0.134991   
ARE          2.135029       1.637240      1.236356      -0.119344   
ALGN         0.204629       0.277842      1.334365      -0.049868   
ALLE        -0.083907       0.084884     -0.028067      -0.175049   
LNT          0.848492       0.192336     -0.002927      -0.147822   
ALL         -0.144815      -0.337025     -0.510712      -0.102444   
GOOGL        0.603841       0.798298      0.552457      -0.041731   
GOOG         0.603841       0.798298      0.552457      -0.041731   
MO           1.846549      -0.068793      0.948950      -0.178179   
AMZN        -0.653571      -0.414579     -0.132610      -0.084919   
AMCR        -0.528022      -0.387438     -0.997468      -0.137495   
AEE          1.091389       0.085627      0.303519      -0.152830   
AAL         -2.508754      -1.467895     -2.089370       0.401723   
AEP          0.698568       0.006959      0.084870      -0.127167   
AXP         -1.410371       0.235010      1.182873      -0.052685   
AIG         -0.168851      -0.263288     -0.540699      -0.130297   
AMT          2.168621       0.688248      1.227702      -0.101505   
AWK          1.410747       0.254306      0.544971      -0.147196   
AMP          0.179915       0.154381      0.378637      -0.181308   
ABC         -1.322883      -0.863736     -1.761289      -0.114649   
AME          0.261861       0.176274     -0.341393      -0.054249   
AMGN         1.283954       0.387308      1.370756      -0.156272   
APH         -0.105002      -0.015782     -0.548358      -0.083354   
ADI          1.135275       0.619759      1.051502      -0.104948   
ANSS         0.534506       0.554132      1.971877      -0.106825   
ANTM        -1.040624      -0.548112     -0.771723      -0.123099   
AON         -0.329689      -0.364590     -0.000114      -0.128419   
AOS         -0.305880      -0.032958     -0.260346      -0.106513   
APA          1.606027      -0.311315      0.802867       0.098785   
AAPL         0.448375       0.605393     -0.085012      -0.079912   
AMAT         0.402736       0.534147      0.130045      -0.041731   
APTV        -0.567270      -0.410285     -1.102876       0.124760   
ADM         -1.136143      -0.602978     -1.580675      -0.042357   
ANET         0.435481       0.733625      0.874221      -0.095872   
AJG         -0.018814      -0.130550     -0.312791      -0.099941   
AIZ         -0.818877      -0.521077     -1.425808      -0.144693   
T            0.317227      -0.731369      0.367344      -0.187880   
ATO          0.926026       0.324544      0.310529      -0.098689   
ADSK        -0.352197       0.892868      2.076723      -0.113085   
ADP          0.046732       0.151359      0.058864      -0.136556   
AZO         -0.086735       0.032563      0.383657      -0.119031   
AVB   

In [117]:
def FunkSVD(fundamentals_mat, latent_features=14, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with symbols as rows, fundamentals as columns and fundamentals as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_symbols = fundamentals_mat.shape[0] # number of rows in the matrix
    n_features = fundamentals_mat.shape[1] # number of movies in the matrix
    num_comb = n_symbols * n_features # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    symbols_mat = np.random.rand(n_symbols, latent_features) # user matrix filled with random values of shape user x latent 
    features_mat = np.random.rand(latent_features, n_features) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    # print("Optimization Statistics")
    # print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for symbol in range(n_symbols):
            for feature in range(n_features):
                # print('This is iteration {}, for user {} and movie {}.'.format(iteration, user, movie))
                
                # if the rating exists
                f_value = fundamentals_mat[symbol][0, feature]
                if not pd.isna(f_value):
                    # print('user {}, movie {}, rating {}'.format(user, movie, rating))
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = f_value - np.dot(symbols_mat[symbol], features_mat[:, feature])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += np.abs(error)
                    
                    # update the values in each matrix in the direction of the gradient
                    updated_symbols_mat = symbols_mat[symbol] + learning_rate * 2 * error * features_mat[:, feature]
                    symbols_mat[symbol] = updated_symbols_mat
                    updated_features_mat = features_mat[:, feature] + learning_rate * 2 * error * symbols_mat[symbol]
                    features_mat[:, feature] = updated_features_mat

        # print results for iteration
        # print('Step: {}, error is: {}.'.format(iteration, sse_accum))
        
    return symbols_mat, features_mat, sse_accum

In [122]:
for learning_rate in (0.005, 0.003, 0.01, 0.0005):
    for nr_iters in (200, 250, 300, 350):
        symbols_mat, features_mat, error = FunkSVD(np.matrix(my_df_extra_normalized), learning_rate=learning_rate, iters=nr_iters)
        print('For learning_rate {} and nr_iters {} error is {}'.format(learning_rate, nr_iters, error))

For learning_rate 0.005 and nr_iters 200 error is 224.80357384694054
For learning_rate 0.005 and nr_iters 250 error is 215.38679792181358
For learning_rate 0.005 and nr_iters 300 error is 210.5512811842937
For learning_rate 0.005 and nr_iters 350 error is 212.3489576252837
For learning_rate 0.003 and nr_iters 200 error is 715.6979355968407
For learning_rate 0.003 and nr_iters 250 error is 635.72926277472
For learning_rate 0.003 and nr_iters 300 error is 544.6212568938921
For learning_rate 0.003 and nr_iters 350 error is 201.62434963975107


C:\Users\Andre\AppData\Local\Temp/ipykernel_28732/693044129.py:56: RuntimeWarning: overflow encountered in multiply
  updated_symbols_mat = symbols_mat[symbol] + learning_rate * 2 * error * features_mat[:, feature]
C:\Users\Andre\AppData\Local\Temp/ipykernel_28732/693044129.py:56: RuntimeWarning: invalid value encountered in add
  updated_symbols_mat = symbols_mat[symbol] + learning_rate * 2 * error * features_mat[:, feature]
C:\Users\Andre\AppData\Local\Temp/ipykernel_28732/693044129.py:58: RuntimeWarning: overflow encountered in multiply
  updated_features_mat = features_mat[:, feature] + learning_rate * 2 * error * symbols_mat[symbol]


For learning_rate 0.01 and nr_iters 200 error is nan
For learning_rate 0.01 and nr_iters 250 error is nan
For learning_rate 0.01 and nr_iters 300 error is nan
For learning_rate 0.01 and nr_iters 350 error is nan
For learning_rate 0.0005 and nr_iters 200 error is 2487.1474981428823
For learning_rate 0.0005 and nr_iters 250 error is 2275.3687722258496
For learning_rate 0.0005 and nr_iters 300 error is 2173.0842525597914
For learning_rate 0.0005 and nr_iters 350 error is 1905.8828358113267


## Learning rate of 0.005 is the most stable one - best outcome for 300 iterations

In [131]:
# now train funk SVD
symbols_mat, features_mat, error = FunkSVD(np.matrix(my_df_extra_normalized), learning_rate=0.005, iters=300)

In [193]:
predicted_df = np.dot(symbols_mat, features_mat)
predicted_df = pd.DataFrame(predicted_df)
predicted_df.index = my_df_extra.index.tolist()

In [194]:
predicted_df.columns = my_df_extra.columns

In [203]:
predicted_df.head()

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
MMM        0.201045       0.107063      0.160589      -0.145871   
ABT        0.241235       0.133542      0.558393      -0.096120   
ABBV       1.539657      -0.063332      1.111823      -0.132505   
ABMD       0.283473       0.063698      1.707734      -0.117673   
ACN       -0.457527      -0.160232     -0.494862      -0.084974   

      operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
MMM           0.208541       -0.075774        0.813549     -0.072709   
ABT           0.119742       -0.059971        0.244152     -0.268537   
ABBV          0.311880       -0.066132        0.373308      1.104500   
ABMD         -0.160811       -0.080344        0.914852     -0.395620   
ACN           0.056840       -0.072102        1.084642     -0.361779   

      returnOnEquity  enterpriseToEbitda  priceToBook  \
MMM         0.020691           -0.261252    -0.135496   
ABT        -0.086613           -0.004754    -0.128225   
ABBV        0.057423           -0.290013     0.183453   
ABMD       -0.150055            0.899122     0.026466   
ACN        -0.042901            0.203682     0.049248   

      priceToSalesTrailing12Months  forwardPE  trailingPE  
MMM                      -0.185300  -0.248630   -0.095041  
ABT                       0.029821   0.012903   -0.062532  
ABBV                     -0.011195  -0.408487   -0.068107  
ABMD                      0.803452   0.907134    0.080005  
ACN                      -0.096301   0.234532   -0.046406

### Example for one symbol

In [139]:
predicted_df.iloc[6] * np.std(my_df_extra, axis=0) + np.mean(my_df_extra, axis=0)

ebitdaMargins                    0.420536
profitMargins                    0.309236
grossMargins                     0.895145
revenueGrowth                    0.183989
operatingMargins                -0.212469
earningsGrowth                  -0.507850
returnOnAssets                   0.141100
debtToEquity                    32.455165
returnOnEquity                   0.313183
enterpriseToEbitda              43.125699
priceToBook                     18.825928
priceToSalesTrailing12Months    14.697810
forwardPE                       35.767691
trailingPE                      58.004762
dtype: float64

In [140]:
my_df_extra.iloc[6]

ebitdaMargins                    0.416810
profitMargins                    0.305480
grossMargins                     0.881850
revenueGrowth                    0.200000
operatingMargins                 0.367560
earningsGrowth                  -0.445000
returnOnAssets                   0.140760
debtToEquity                    31.581000
returnOnEquity                   0.343680
enterpriseToEbitda              41.021000
priceToBook                     18.347166
priceToSalesTrailing12Months    17.191498
forwardPE                       35.097847
trailingPE                      56.920160
Name: ADBE, dtype: float64

In [159]:
# create a df for std and mean which can be imported for flask app and make predictions
mean_std_funkSVD = pd.DataFrame()
mean_std_funkSVD = mean_std_funkSVD.append(np.std(my_df_extra, axis=0), ignore_index=True)
mean_std_funkSVD = mean_std_funkSVD.append(np.mean(my_df_extra, axis=0), ignore_index=True)
mean_std_funkSVD.index = ['std', 'mean']

In [160]:
mean_std_funkSVD

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
std        0.176824       0.188642      0.231101       3.195366   
mean       0.249387       0.144617      0.437436       0.543347   

      operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
std           2.151249       42.430729        0.056138    393.144484   
mean          0.051880        3.274123        0.063625    157.237970   

      returnOnEquity  enterpriseToEbitda  priceToBook  \
std         2.229793           39.462324    32.734389   
mean        0.420320           20.782879    11.364808   

      priceToSalesTrailing12Months  forwardPE  trailingPE  
std                      10.394784   48.03494  536.776285  
mean                      5.991150   28.60953   68.294307

In [149]:
np.mean(my_df_extra, axis=0)

ebitdaMargins                     0.249387
profitMargins                     0.144617
grossMargins                      0.437436
revenueGrowth                     0.543347
operatingMargins                  0.051880
earningsGrowth                    3.274123
returnOnAssets                    0.063625
debtToEquity                    157.237970
returnOnEquity                    0.420320
enterpriseToEbitda               20.782879
priceToBook                      11.364808
priceToSalesTrailing12Months      5.991150
forwardPE                        28.609530
trailingPE                       68.294307
dtype: float64

## Save predicted_df, mean and std in case someone wants to predict missing fundamentals for S&P 500 index, we don't have to retrain funk SVD model

In [248]:
predicted_df.to_csv('predicted_df', sep='\t', index_label='Symbol')

In [239]:
mean_std_funkSVD.to_csv('mean_std_funkSVD', sep='\t', index_label='Statistic')

## Test for 1 example:

In [240]:
a = pd.read_csv('mean_std_funkSVD', sep='\t')

In [242]:
a = a.set_index(['Statistic'])

In [243]:
a

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
Statistic                                                              
std             0.176824       0.188642      0.231101       3.195366   
mean            0.249387       0.144617      0.437436       0.543347   

           operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
Statistic                                                                   
std                2.151249       42.430729        0.056138    393.144484   
mean               0.051880        3.274123        0.063625    157.237970   

           returnOnEquity  enterpriseToEbitda  priceToBook  \
Statistic                                                    
std              2.229793           39.462324    32.734389   
mean             0.420320           20.782879    11.364808   

           priceToSalesTrailing12Months  forwardPE  trailingPE  
Statistic                                                       
std                           10.394784   48.03494  536.776285  
mean                           5.991150   28.60953   68.294307

In [249]:
b = pd.read_csv('predicted_df', sep='\t')

In [250]:
b = b.set_index(['Symbol'])

In [251]:
b

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
Symbol                                                              
MMM          0.201045       0.107063      0.160589      -0.145871   
ABT          0.241235       0.133542      0.558393      -0.096120   
ABBV         1.539657      -0.063332      1.111823      -0.132505   
ABMD         0.283473       0.063698      1.707734      -0.117673   
ACN         -0.457527      -0.160232     -0.494862      -0.084974   
ATVI         0.714289       0.768222      1.230312      -0.150006   
ADBE         0.967912       0.872649      1.980553      -0.112462   
AMD         -0.094399       0.661165      0.168309      -0.004317   
AAP         -0.800015      -0.473070      0.027688      -0.157823   
AES          0.534185      -0.516817     -0.767564      -0.108790   
AFL         -0.012932       0.211990     -0.154948      -0.192316   
A            0.142762       0.244365      0.447190      -0.133596   
APD          0.612998       0.314749     -0.555267      -0.101685   
AKAM         0.473320       0.161569      0.835490      -0.142592   
ALK         -1.817932      -0.747760     -1.595864       0.386769   
ALB         -0.013923      -0.417962     -0.512693      -0.138214   
ARE          2.155409       1.651099      1.285893      -0.123762   
ALGN         0.221696       0.284460      1.372114      -0.053400   
ALLE        -0.084302       0.070358     -0.044003      -0.174133   
LNT          0.848045       0.182095     -0.012555      -0.147209   
ALL         -0.145995      -0.353397     -0.528794      -0.101184   
GOOGL        0.612635       0.793157      0.559983      -0.042871   
GOOG         0.610579       0.792615      0.558397      -0.042717   
MO           1.845122      -0.088295      0.928740      -0.177061   
AMZN        -0.653865      -0.431514     -0.149724      -0.083888   
AMCR        -0.524870      -0.400834     -1.006473      -0.137163   
AEE          1.089759       0.068140      0.285655      -0.151747   
AAL         -2.482932      -1.446711     -2.013556       0.396036   
AEP          0.693353      -0.008581      0.060587      -0.125388   
AXP         -1.411973       0.215401      1.161454      -0.051190   
AIG         -0.170032      -0.278143     -0.556366      -0.129223   
AMT          2.187681       0.693599      1.267274      -0.105367   
AWK          1.420619       0.253251      0.562887      -0.149109   
AMP          0.177879       0.135901      0.360670      -0.180151   
ABC         -1.320854      -0.878745     -1.777407      -0.114386   
AME          0.272976       0.176221     -0.322117      -0.056287   
AMGN         1.279816       0.361867      1.337051      -0.154295   
APH         -0.093809      -0.020897     -0.535159      -0.085108   
ADI          1.147021       0.616294      1.069632      -0.107057   
ANSS         0.564403       0.572930      2.044907      -0.113453   
ANTM        -1.039409      -0.565675     -0.786799      -0.122470   
AON         -0.322733      -0.376753     -0.000764      -0.129126   
AOS         -0.301928      -0.048824     -0.269291      -0.106638   
APA          1.592975      -0.354364      0.733701       0.103413   
AAPL         0.460350       0.599124     -0.074545      -0.081803   
AMAT         0.412486       0.522418      0.132250      -0.043066   
APTV        -0.560196      -0.418089     -1.100541       0.123962   
ADM         -1.127980      -0.616035     -1.581283      -0.043130   
ANET         0.429472       0.702636      0.830185      -0.093370   
AJG         -0.009442      -0.144343     -0.312421      -0.100972   
AIZ         -0.812436      -0.528781     -1.421579      -0.145575   
T            0.315012      -0.758839      0.332344      -0.185970   
ATO          0.928348       0.305329      0.294632      -0.098347   
ADSK        -0.343314       0.886886      2.089098      -0.114790   
ADP          0.058636       0.145190      0.071780      -0.138457   
AZO         -0.092660       0.004011      0.339801      -0.116541   
AVB   

In [256]:
mean_funkSVD = np.array(a.iloc[1])
std_funkSVD = np.array(a.iloc[0])

In [254]:
predicted_df

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
MMM         0.201045       0.107063      0.160589      -0.145871   
ABT         0.241235       0.133542      0.558393      -0.096120   
ABBV        1.539657      -0.063332      1.111823      -0.132505   
ABMD        0.283473       0.063698      1.707734      -0.117673   
ACN        -0.457527      -0.160232     -0.494862      -0.084974   
ATVI        0.714289       0.768222      1.230312      -0.150006   
ADBE        0.967912       0.872649      1.980553      -0.112462   
AMD        -0.094399       0.661165      0.168309      -0.004317   
AAP        -0.800015      -0.473070      0.027688      -0.157823   
AES         0.534185      -0.516817     -0.767564      -0.108790   
AFL        -0.012932       0.211990     -0.154948      -0.192316   
A           0.142762       0.244365      0.447190      -0.133596   
APD         0.612998       0.314749     -0.555267      -0.101685   
AKAM        0.473320       0.161569      0.835490      -0.142592   
ALK        -1.817932      -0.747760     -1.595864       0.386769   
ALB        -0.013923      -0.417962     -0.512693      -0.138214   
ARE         2.155409       1.651099      1.285893      -0.123762   
ALGN        0.221696       0.284460      1.372114      -0.053400   
ALLE       -0.084302       0.070358     -0.044003      -0.174133   
LNT         0.848045       0.182095     -0.012555      -0.147209   
ALL        -0.145995      -0.353397     -0.528794      -0.101184   
GOOGL       0.612635       0.793157      0.559983      -0.042871   
GOOG        0.610579       0.792615      0.558397      -0.042717   
MO          1.845122      -0.088295      0.928740      -0.177061   
AMZN       -0.653865      -0.431514     -0.149724      -0.083888   
AMCR       -0.524870      -0.400834     -1.006473      -0.137163   
AEE         1.089759       0.068140      0.285655      -0.151747   
AAL        -2.482932      -1.446711     -2.013556       0.396036   
AEP         0.693353      -0.008581      0.060587      -0.125388   
AXP        -1.411973       0.215401      1.161454      -0.051190   
AIG        -0.170032      -0.278143     -0.556366      -0.129223   
AMT         2.187681       0.693599      1.267274      -0.105367   
AWK         1.420619       0.253251      0.562887      -0.149109   
AMP         0.177879       0.135901      0.360670      -0.180151   
ABC        -1.320854      -0.878745     -1.777407      -0.114386   
AME         0.272976       0.176221     -0.322117      -0.056287   
AMGN        1.279816       0.361867      1.337051      -0.154295   
APH        -0.093809      -0.020897     -0.535159      -0.085108   
ADI         1.147021       0.616294      1.069632      -0.107057   
ANSS        0.564403       0.572930      2.044907      -0.113453   
ANTM       -1.039409      -0.565675     -0.786799      -0.122470   
AON        -0.322733      -0.376753     -0.000764      -0.129126   
AOS        -0.301928      -0.048824     -0.269291      -0.106638   
APA         1.592975      -0.354364      0.733701       0.103413   
AAPL        0.460350       0.599124     -0.074545      -0.081803   
AMAT        0.412486       0.522418      0.132250      -0.043066   
APTV       -0.560196      -0.418089     -1.100541       0.123962   
ADM        -1.127980      -0.616035     -1.581283      -0.043130   
ANET        0.429472       0.702636      0.830185      -0.093370   
AJG        -0.009442      -0.144343     -0.312421      -0.100972   
AIZ        -0.812436      -0.528781     -1.421579      -0.145575   
T           0.315012      -0.758839      0.332344      -0.185970   
ATO         0.928348       0.305329      0.294632      -0.098347   
ADSK       -0.343314       0.886886      2.089098      -0.114790   
ADP         0.058636       0.145190      0.071780      -0.138457   
AZO        -0.092660       0.004011      0.339801      -0.116541   
AVB         1.961995       1.798188      0.850129      -0.183927   
AVY        -0.519480      -0.302363     -0.696300      -0.10809

In [261]:
# predicted
pd.DataFrame((predicted_df.loc['XOM'] * std_funkSVD + mean_funkSVD)).transpose()

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
XOM       0.114796      -0.062614      0.309439       1.030739   

     operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
XOM          0.168093       -0.373814        0.010847     31.280627   

     returnOnEquity  enterpriseToEbitda  priceToBook  \
XOM       -0.054693           11.118637     2.053475   

     priceToSalesTrailing12Months  forwardPE  trailingPE  
XOM                      1.760208  14.427279   -1.193511

In [262]:
# actual
pd.DataFrame(my_df_extra.loc['XOM']).transpose()

ebitdaMargins  profitMargins  grossMargins  revenueGrowth  \
XOM        0.11325       -0.06175       0.31561           1.04   

     operatingMargins  earningsGrowth  returnOnAssets  debtToEquity  \
XOM           0.02937             0.0         0.01134        36.611   

     returnOnEquity  enterpriseToEbitda  priceToBook  \
XOM        -0.07788              12.393     1.651879   

     priceToSalesTrailing12Months  forwardPE  trailingPE  
XOM                      1.201156  13.641426         0.0